Import numpy library and create a random matrix

In [1]:
import numpy as np
n = 4
A = np.random.random((n,n))
A

array([[ 0.49922613,  0.07979019,  0.23518855,  0.1647902 ],
       [ 0.74323602,  0.04472799,  0.54030603,  0.37305325],
       [ 0.40874152,  0.51150855,  0.17782078,  0.6479568 ],
       [ 0.57472887,  0.67596946,  0.73410843,  0.97596626]])

Create another matrix and multiply the two

In [2]:
B = np.random.random((n,n))
np.dot(A,B)

array([[ 0.75001965,  0.43859658,  0.73518856,  0.83285544],
       [ 1.27756494,  0.72708474,  1.27528118,  1.39893694],
       [ 1.16023982,  0.85408688,  1.17364888,  1.42447064],
       [ 1.90054979,  1.44349232,  1.91599113,  2.32447089]])

Import Cyclops Tensor Framework library and convert numpy matrices to CTF matrices

In [3]:
import ctf
tA = ctf.astensor(A)
tB = ctf.astensor(B)
ctf.dot(tA,tB)

array([[ 0.75001965,  0.43859658,  0.73518856,  0.83285544],
       [ 1.27756494,  0.72708474,  1.27528118,  1.39893694],
       [ 1.16023982,  0.85408688,  1.17364888,  1.42447064],
       [ 1.90054979,  1.44349232,  1.91599113,  2.32447089]])

Use CTF index-based notation to perform multiplication

In [4]:
tC = ctf.zeros((n,n))
tC.i("ij") << tA.i("ik")*tB.i("kj")
tC

array([[ 0.75001965,  0.43859658,  0.73518856,  0.83285544],
       [ 1.27756494,  0.72708474,  1.27528118,  1.39893694],
       [ 1.16023982,  0.85408688,  1.17364888,  1.42447064],
       [ 1.90054979,  1.44349232,  1.91599113,  2.32447089]])

The particular character `'i','j','k'` don't matter, we can replace them with `'z','?','+'`

In [5]:
tC = ctf.zeros((n,n))
tC.i("z?") << tA.i("z+")*tB.i("+?")
tC

array([[ 0.75001965,  0.43859658,  0.73518856,  0.83285544],
       [ 1.27756494,  0.72708474,  1.27528118,  1.39893694],
       [ 1.16023982,  0.85408688,  1.17364888,  1.42447064],
       [ 1.90054979,  1.44349232,  1.91599113,  2.32447089]])

`numpy` actually has similar functionality via `einsum`

In [6]:
np.einsum("ik,kj->ij",A,B)

array([[ 0.75001965,  0.43859658,  0.73518856,  0.83285544],
       [ 1.27756494,  0.72708474,  1.27528118,  1.39893694],
       [ 1.16023982,  0.85408688,  1.17364888,  1.42447064],
       [ 1.90054979,  1.44349232,  1.91599113,  2.32447089]])

In [7]:
ctf.einsum("ik,kj->ij",tA,tB)

array([[ 0.75001965,  0.43859658,  0.73518856,  0.83285544],
       [ 1.27756494,  0.72708474,  1.27528118,  1.39893694],
       [ 1.16023982,  0.85408688,  1.17364888,  1.42447064],
       [ 1.90054979,  1.44349232,  1.91599113,  2.32447089]])

This notation can be used to contract tensor networks, for instance the tensor train (MPS)

In [8]:
k = 2 # rank
W = ctf.tensor([k,n,k])
V = ctf.tensor([k,n])
W.fill_random(-1.0,1.0)
V.fill_random(-1.0,1.0)
Z = ctf.tensor([n,n,n,n,n,n])
Z.i("ijklmn") << V.i("ai")*W.i("ajb")*W.i("bkc")*W.i("cld")*W.i("dme")*V.i("en")
print(Z[1:3,0,2,1,0:3,1].reshape((3,2)))



array([[ 0.07366687,  0.12803461],
       [ 0.06423109,  0.02295772],
       [ 0.05320817,  0.02112921]])


Using `np.einsum` the contractions look as follows

In [11]:
V2 = V.to_nparray()
W2 = W.to_nparray()
Z2 = np.einsum("ai,ajb,bkc,cld,dme,en->ijklmn",V2,W2,W2,W2,W2,V2)
print(Z2[1:3,0,2,1,0:3,1].reshape((3,2)))

#same possible with CTF
Z = ctf.einsum("ai,ajb,bkc,cld,dme,en->ijklmn",V,W,W,W,W,V)
print(Z[1:3,0,2,1,0:3,1].reshape((3,2)))

[[ 0.07366687  0.12803461]
 [ 0.06423109  0.02295772]
 [ 0.05320817  0.02112921]]
array([[ 0.07366687,  0.12803461],
       [ 0.06423109,  0.02295772],
       [ 0.05320817,  0.02112921]])


To contract together a CP decomposition, we need to use Hadamard products

In [13]:
U = ctf.tensor([k,n])
U.fill_random(-1.0,1.0)

Z.set_zero()

#note that the `a` index appears in multiple operands
Z.i("ijklmn") << U.i("ai")*U.i("aj")*U.i("ak")*U.i("al")*U.i("am")*U.i("an")
print(Z[1:3,0,2,1,0:3,1].reshape((3,2)))

U2 = U.to_nparray()
Z2 = np.einsum("ai,aj,ak,al,am,an->ijklmn",U2,U2,U2,U2,U2,U2)
print(Z2[1:3,0,2,1,0:3,1].reshape((3,2)))


array([[-0.06735958,  0.01278285],
       [-0.05052931,  0.02043648],
       [-0.05052931,  0.04237852]])
[[-0.06735958  0.01278285]
 [-0.05052931  0.02043648]
 [-0.05052931  0.04237852]]


Lets test the preformance of the MPS contractions for a different rank

In [16]:
import time
for k in np.arange(4)*2+2:
    t = time.time()
    W = ctf.tensor([k,n,k])
    V = ctf.tensor([k,n])
    W.fill_random(-1.0,1.0)
    V.fill_random(-1.0,1.0)
    Z = ctf.tensor([n,n,n,n,n,n])
    Z.i("ijklmn") << V.i("ai")*W.i("ajb")*W.i("bkc")*W.i("cld")*W.i("dme")*V.i("en")
    V2 = V.to_nparray()
    W2 = W.to_nparray()
    #time CTF, including all initialization and conversions
    print("ctf   k =",k,"took",time.time()-t,"seconds.")
    t2 = time.time()
    Z2 = np.einsum("ai,ajb,bkc,cld,dme,en->ijklmn",V2,W2,W2,W2,W2,V2)
    print("numpy k =",k,"took",time.time()-t2,"seconds.")

ctf   k = 2 took 0.028398990631103516 seconds.
numpy k = 2 took 0.012132644653320312 seconds.
ctf   k = 4 took 0.17014527320861816 seconds.
numpy k = 4 took 0.22895264625549316 seconds.
ctf   k = 6 took 0.88639235496521 seconds.
numpy k = 6 took 1.2816572189331055 seconds.
ctf   k = 8 took 4.034878253936768 seconds.
numpy k = 8 took 4.610719680786133 seconds.


Lets create a sparse tensor of total size $4^{12}$

In [17]:
Z = ctf.tensor([n,n,n,n,n,n,n,n,n,n,n,n],sp=1)
Z.fill_sp_random(-1.,1.,.00001)
Z.read_local_nnz()

(array([   22719,    93826,   119244,   267725,   487669,   660105,
          695677,   874440,   901502,   950291,  1007954,  1066395,
         1129878,  1143853,  1151885,  1220807,  1719826,  1763154,
         1789555,  1867359,  1973550,  2109998,  2130630,  2386222,
         2578764,  2696878,  2741576,  2843608,  3136093,  3228922,
         3596926,  3702374,  3710983,  3786229,  4054600,  4095468,
         4347769,  4359880,  4450764,  4461709,  4707221,  4863154,
         4870663,  4877028,  5190057,  5348657,  5416971,  5641826,
         5852037,  5919829,  5969084,  6000715,  6221829,  6442957,
         6605931,  6648639,  6760104,  6791844,  6920001,  7004972,
         7012191,  7088057,  7196409,  7229832,  7344236,  7432969,
         7564478,  7588891,  7738676,  7760252,  7999727,  8086936,
         8183463,  8208168,  8213808,  8220184,  8247167,  8387430,
         8420390,  8512627,  8705658,  8752247,  8873669,  8882706,
         8910018,  8989521,  9222430,  9287663, 

In [18]:
#create a random vector in a sparse representation
v = ctf.tensor([n],sp=1)
v.fill_sp_random(0.,1.,1.)

#create an order 12 sparse tensor
Z = ctf.tensor([n,n,n,n,n,n,n,n,n,n,n,n],sp=1)

#fill tensor so that .001% of entries are nonzero
Z.fill_sp_random(0.,1.,.00001)

#set diagonal to zero
Z.i("iiiiiiiiiiii") << 1. 

str12 = "1234567890ab"
for i in range(1,12)[::-1]:
    #normalize tensor
    Z.i(str12[0:i]).scl(1./ctf.sum(Z))
    #create tensor with one less dimension
    Z_new = ctf.tensor([n for j in range(i)],sp=1)
    #contract tensor over its last mode with a vector
    Z_new.i(str12[0:i]) << Z.i(str12[0:i+1])*v.i(str12[i])
    #replace old tensor with lower-dimensional one
    Z = Z_new

#read the nonzeros from Z stored on this processor
inds, vals = Z.read_local_nnz()
print(Z.ndim)
print(inds,vals)

1
[0 1 2 3] [ 0.00282648  0.02617537  0.54507062  0.00662197]
